In [59]:
import pandas as pd

df = pd.read_csv("../ds_salaries.csv")

display(df)

print(df["company_size"].unique())

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


['L' 'S' 'M']


In [60]:
df_features = df.copy(deep=True)

target = df_features["salary_in_usd"]
df_features.drop(["salary", "salary_currency", "salary_in_usd"], axis=1, inplace=True)

display(df_features)

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,ES,100,ES,L
1,2023,MI,CT,ML Engineer,US,100,US,S
2,2023,MI,CT,ML Engineer,US,100,US,S
3,2023,SE,FT,Data Scientist,CA,100,CA,M
4,2023,SE,FT,Data Scientist,CA,100,CA,M
...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,US,100,US,L
3752,2020,EN,FT,Data Scientist,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,US,100,US,L


# Create preprocessor

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

categorical_encoder = Pipeline([("categorical", OneHotEncoder())])
ordinal_encoder = Pipeline([("ordinal", OrdinalEncoder())])

# define ColumnTransformer for different features
preprocessesor = ColumnTransformer(
    transformers = [
        ('cat', categorical_encoder, ["employment_type", "job_title", "employee_residence", "remote_ratio", "company_location"]),
        ('ord', ordinal_encoder, ["company_size", "work_year", "experience_level"]),
    ]
)

preprocessesor.set_output(transform="pandas")
preprocessed_df = preprocessesor.fit_transform(df_features)
display(preprocessed_df)

<3755x253 sparse matrix of type '<class 'numpy.float64'>'
	with 29190 stored elements in Compressed Sparse Row format>

# init Model

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

estimator = LinearRegression()

# Execute pipeline and evaluate

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

steps = [('estimator', estimator)]

pipeline = Pipeline(steps)

kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

r2_scores = cross_val_score(pipeline, preprocessed_df, target, cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, preprocessed_df, target, cv=kfold, scoring='neg_mean_squared_error')


print(np.mean(r2_scores))
print(np.mean(mse_scores))

0.3594218793998336
2544087654.4091425
